# TES biasing (Ari)

You will find that the names of TES-biasing functions seem arcane and convoluted. This is due to the organic evolution of the software as subtle changes to the functionality became necessary. The overall organization of the software has been stable for some time now, so the names of these functions are likely to change in the near future to be simpler and more self-explanatory.

## Setting up

This notebook assumes that resonators have been tuned, flux ramp is on and tone tracking is on.

Import pysmurf and instantiate a pysmurf object. Specify your configuration file (cfg_file). Since we are working interactively, set make_logfile to False. This will print messages to the terminal rather than just to a log file.

In [ ]:
import pysmurf
cfg_file='/usr/local/controls/Applications/smurf/pysmurf/pysmurf/cfg_files/experiment_fp28_smurfsrv04.cfg'
S = pysmurf.SmurfControl(make_logfile=False,cfg_file=cfg_file)

## Setting bias voltages

Check current state of bias voltages.

In [ ]:
S.get_tes_bias_bipolar_array()

Turn off bias voltages.

In [ ]:
S.set_tes_bias_off()

Manually setting a TES bias on, e.g., group 3. Then check that the bias voltage has been updated.

In [ ]:
S.set_tes_bias_bipolar(3, 1.2)
S.get_tes_bias_bipolar_array()

Or set TES biases on all groups at once.

In [ ]:
bias_array = np.zeros(8)
bias_array[3] = 1.2
S.set_tes_bias_bipolar_array(bias_array)
S.get_tes_bias_bipolar_array()

Overbias to the voltage set by overbias_voltage, and then drop to a bias point set by tes_bias. Use group 3 as an example. The overbias voltage is always in high-current mode. The TES bias point can be chosen to be in either high- or low-current mode. For the current system at Stanford. It is more convenient to operate the TESs in low-current mode, which also has better filtering on the bias lines. 

After dropping to a bias point, check again that the bias voltage has been set correctly.

In [ ]:
S.overbias_tes(3,overbias_voltage=15.,tes_bias=3.4,high_current_mode=False)
S.get_tes_bias_bipolar_array()

## IV curves

Take IV curves on chosen bias groups. The Stanford system currently only has TESs on group 3, but we can run the code on multiple groups anyway.

Set bias_groups to be the list of groups for which you wish to measure IV curves. Choose whether to run the IV in high- or low-current mode with the boolean high_current_mode. On the Stanford system, it is possible to run in either mode. The IV curve runs from a high voltage down to a low voltage. The default low voltage is 0, but it can be changed with the default argument bias_low. The upper end of the IV curve is controlled by bias_high. The voltage is decreased from bias_high to bias_low in steps of size bias_step. To kick the TESs out of the superconducting state to begin the IV curve, a large voltage is supplied that is set by overbias_voltage. The TESs are then dropped down to bias_high (in either high- or low-current mode depending on the boolean high_current_mode), and we wait for a time specified by the optional argument cool_wait (seconds) before beginning the IV curve.

The IV script automatically analyzes the data and makes plots. Many channels will produce little more than noise, though, since they are either not connected to TESs or not on the active bias groups. To avoid wasting time on the analysis of unresponsive channels, we set a threshold phase excursion, i.e., a minimum amount that the flux-ramp phase, our raw data, has to change by, and don't analyze channels that move by a smaller amount. This threshold is controlled by the parameter phase_excursion_min and is set in radians. If we refer 1 radian of flux-ramp phase shift to a physical TES current, we find that it is approximately equivalent, within an O(1) factor, to 1 uA.

In [ ]:
S.slow_iv_all(bias_groups=[1,2,3],bias_high=1.5,bias_low=0.,bias_step=0.005,high_current_mode=True,overbias_voltage=15.,cool_wait=30.)

The IV script writes the raw data to an output directory associated with this particular instance of the pysmurf object. You can always find it by running the following command:

In [ ]:
S.output_dir

The data files are tagged with the C time in seconds, in this case given as a ten-digit integer. All outputs associated with this IV measurement will have files names that begin with the same tag. 

In the output directory, you will see a newly created .dat file. This contains the raw timestreams from the IV measurement. You will also see several .npy files with the same tag. The file with a name ending in "iv_bias_all.npy" contains information on the bias points set during the measurement. The file ending in "iv_raw_data.npy" contains metadata that is useful for analyzing and even reanalyzing the IV measurements.

### IV analysis

When the analysis is complete, the output directory will contain a file ending in "iv.npy". This file contains all the derived electrical quantities, in particular, the IV relation.

The analysis will have also created plots for each channel that met the phase-excursion threshold. Every instance of a pysmurf object has an associated plot directory, which you can find by running the following command:

In [ ]:
S.plot_dir

In the plot directory, you will find a number of .png files that all have names beginning with the same c-time tag as the data files. The files with names that contain the string "IV_stream" show the raw data, i.e., flux-ramp phase vs. sample number. For TES or resistor channels, you should already be able to see an IV curve, though there may be a sign flip relative to your expectation. You also may be able to see discrete discrete steps, which occur every time the bias voltage is stepped down. The analysis avoids these transient responses. 

The samples taken at each bias point are averaged to get the phase at each bias voltage. The phase is then converted to a TES current. The analysis then fits the normal portion to get the normal resistance and the correct offset in IV space. What we actually plot is not I vs. V, though. We plot TES current vs. "bias current", by which we mean the current flowing on the TES bias lines through the shunt-resistor network. The shunt-resistor network is current biased, so the "bias current" is a known quantity as soon as a voltage is set on the bias lines, which we tend to call the "commanded voltage". When the TES goes superconducting, we expect the TES current to be exactly equal to the bias current, so we should have a slope of 1 in this portion. Typically, the tracking algorithm does not follow the fast jump in current that occurs when the TES goes fully superconducting, so we insert an offset to the superconducting branch in the analysis to make sure it passes through the origin. This is a bit artificial but has no operational consequences.

The analyzed IV plots have filenames containing the string "IV_curve". Each PNG contains several subplots. The three on the left are all a function of bias current. The top shows TES current and will look like a familiar Nike logo. The middle shows fractional resistance. The bottom shows the estimated power-to-current responsivity (assuming beta = 0). The shaded red area is meant to indicate the transition region. On the right is a plot of fractional resistance as a function of electrical power on the TES. The green dashed line and green dot indicate a target operating resistance.

The IV analysis is summarized in a histogram contained in the file with name containing string "IV_hist.png".

## Noise vs. bias voltage